In [1]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import time
import calendar
from datetime import date
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementNotInteractableException
import ipyparallel as ipp

In [2]:
products=['TV']

In [3]:
url='https://www.amazon.in/'
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get(url)

In [4]:
ratings_5=[]
ratings_4=[]
ratings_3=[]
ratings_2=[]
ratings_1=[]

In [5]:
def change_product(i):
    search=driver.find_element_by_xpath("//input[@id='twotabsearchtextbox']")
    search_btn=driver.find_element_by_xpath("//input[@id='nav-search-submit-button']")
    search.clear()
    search.send_keys(i)
    search_btn.click()
    time.sleep(3)

In [6]:
def find_product_links():
    linkss=[]
    urls=driver.find_elements_by_xpath('//a[@class="a-link-normal s-link-style a-text-normal"]')
    for i in urls:
        linkss.append(i.get_attribute('href'))
    return linkss

In [7]:
def item_review_nav():
    button_read_more=driver.find_element_by_xpath('//*[@id="reviews-medley-footer"]/div[2]/a')
    button_read_more.click()
    time.sleep(3)

In [8]:
def item_review_all():
    review=driver.find_elements_by_xpath('//a[@class="a-size-base a-link-normal see-all"]')
    positive=0
    critical=0
    for i in review:
        if 'positive' in i.text.split():
            positive=i
        else:
            critical=i
    return positive,critical

In [9]:
def populate_data(rating_5,rating_4,rating_3,rating_2,rating_1):
    rating=driver.find_elements_by_xpath('//div[@class="a-row"]/a[@class="a-link-normal"]')
    comments=driver.find_elements_by_xpath('//a[@class="a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"]')
    for i in range(len(rating)):
        if len(rating[i].get_attribute('title'))>0:
            rate=int(float(rating[i].get_attribute('title').split()[0]))
            if rate==5 and len(rating_5)<2500:
                rating_5.append(comments[i].text)
            elif rate==4 and len(rating_4)<2500:
                rating_4.append(comments[i].text)
            elif rate==3 and len(rating_3)<2500:
                rating_3.append(comments[i].text)
            elif rate==2 and len(rating_2)<2500:
                rating_2.append(comments[i].text)
            elif rate==1 and len(rating_1)<2500:
                rating_1.append(comments[i].text)
    return rating_5,rating_4,rating_3,rating_2,rating_1

In [10]:
for i in products:
    print(i)
    change_product(i)
    rating_5=[]
    rating_4=[]
    rating_3=[]
    rating_2=[]
    rating_1=[]
    while len(rating_1)<2500 or len(rating_2)<2500 or len(rating_3)<2500 or len(rating_4)<2500 or len(rating_5)<2500:
        try:
            nexty=driver.find_element_by_xpath('//div[@class="a-section a-spacing-large a-spacing-top-large a-text-center s-pagination-container"]//a')
        except:
            try:
                nexty=driver.find_element_by_xpath('//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
            except:
                nexty=driver.find_element_by_xpath('//li[@class="a-last"]/a')
        page=nexty.get_attribute('href')
        links=find_product_links()
        for linky in links:
            driver.get(linky)
            time.sleep(3)
            item_review_nav()
            positive,critical=item_review_all()
            try:
                critical.click()
                time.sleep(3)
                while True:
                    rating_5,rating_4,rating_3,rating_2,rating_1=populate_data(rating_5,rating_4,rating_3,rating_2,rating_1)
                    navy=driver.find_element_by_xpath('//li[@class="a-last"]/a')
                    navi=navy.get_attribute('href')
                    driver.get(navi)
                    time.sleep(3)
            except:
                pass
            positive,critical=item_review_all()
            positive.click()
            time.sleep(3)
            try:
                while True:
                    rating_5,rating_4,rating_3,rating_2,rating_1=populate_data(rating_5,rating_4,rating_3,rating_2,rating_1)
                    navy=driver.find_element_by_xpath('//li[@class="a-last"]/a')
                    navi=navy.get_attribute('href')
                    driver.get(navi)
                    time.sleep(3)
            except:
                pass
            print(len(rating_5),len(rating_4),len(rating_3),len(rating_2),len(rating_1))
            if len(rating_1)>=2500 and len(rating_2)>=2500 and len(rating_3)>=2500 and len(rating_4)>=2500 and len(rating_5)>=2500:
                break
    ratings_1.extend(rating_1)
    ratings_2.extend(rating_2)
    ratings_3.extend(rating_3)
    ratings_4.extend(rating_4)
    ratings_5.extend(rating_5)
    driver.get(page)
    time.sleep(3)

TV
343 123 53 37 186
424 154 64 50 219
1934 843 298 187 725
2500 1972 868 529 2311
2500 2500 1438 871 2500
2500 2500 2112 1166 2500
2500 2500 2352 1273 2500
2500 2500 2500 1452 2500
2500 2500 2500 1747 2500
2500 2500 2500 1926 2500
2500 2500 2500 2063 2500
2500 2500 2500 2139 2500
2500 2500 2500 2211 2500
2500 2500 2500 2348 2500
2500 2500 2500 2500 2500


In [11]:
phone_5=pd.DataFrame({})
phone_5['comments']=ratings_5[:2500]
phone_5['rating']=[5 for i in range(2500)]
phone_4=pd.DataFrame({})
phone_4['comments']=ratings_4[:2500]
phone_4['rating']=[4 for i in range(2500)]
phone_3=pd.DataFrame({})
phone_3['comments']=ratings_3[:2500]
phone_3['rating']=[3 for i in range(2500)]
phone_2=pd.DataFrame({})
phone_2['comments']=ratings_2[:2500]
phone_2['rating']=[2 for i in range(2500)]
phone_1=pd.DataFrame({})
phone_1['comments']=ratings_1[:2500]
phone_1['rating']=[1 for i in range(2500)]

In [12]:
flipkart_phone=phone_5.copy()
flipkart_phone=flipkart_phone.append(phone_4,ignore_index = True)
flipkart_phone=flipkart_phone.append(phone_3,ignore_index = True)
flipkart_phone=flipkart_phone.append(phone_2,ignore_index = True)
flipkart_phone=flipkart_phone.append(phone_1,ignore_index = True)
flipkart_phone

,comments,rating
0,"Spectacular Specification, Build Quality, and ...",5
1,Speaker is ok ok,5
2,HAPPY And SUPER HAPPY to buy this TV,5
3,Glorious 4K with Oodles of Performance for the...,5
4,Cheap and best..... Worth buying,5
...,...,...
12495,Don't fall for the Brand Name,1
12496,Worst Experience Fraud dealer,1
12497,Very very very bad Amazon,1
12498,Smart tv with non-functioning smart features.,1


In [13]:
flipkart_phone.to_csv('amazon_TV.csv',index=False)